In [2]:
!pip install openai --quiet
!pip install mosestokenizer --quiet
!pip install nltk --quiet

In [3]:
import config
import json

In [4]:
import openai

openai.api_key = config.api_key

In [5]:
# 'Substitute offensive/hate terms in the tweet with inclusive language and generate JSON of a few rephrased versions of the tweet, and list down the suspected problematic terms:'

In [6]:
# 09 and 10 June 2023, UTC based

In [7]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
import mosestokenizer
import numpy as np

In [8]:
df = pd.read_csv(config.LSHS_DATAFILE, index_col=0)

In [9]:
df.shape

(55660, 3)

In [10]:
df.head()

,TweetID,Tweet,Label
Domain,,,
Gender,1344706773245038592,WATCH: Video previews #SurreyBC-shot film to f...,0
Gender,1344706877217792005,Men and women don’t have to solve their proble...,0
Gender,1344707261155962880,"At last I awake, very queer about the head, as...",0
Gender,1344707529213792256,WATCH: Video previews Surrey-shot film to focu...,0
Gender,1344709019865403394,heteronormativity is killing my people. how so...,1


In [11]:
df['Label'].value_counts()

Label
0    44874
1     9669
2     1117
Name: count, dtype: int64

In [12]:
tk = TweetTokenizer()
detk = mosestokenizer.MosesDetokenizer('en')
t = df.iloc[3390].to_list()[1]
tk = tk.tokenize(t)
display(t)
detk(tk)

"MS-13 gang member charged in gang's latest attack on transgender women in LA's MacArthur Park https://t.co/AA2asFViuh"

"MS - 13 gang member charged in gang's latest attack on transgender women in LA's MacArthur Park https://t.co/AA2asFViuh"

In [13]:
def get_offensive_words():
    _df = pd.read_csv(config.en_swear_words_datafile, index_col=0)
    
    s = np.logical_or(_df['Level of offensiveness']=='Strongest words', _df['Level of offensiveness']=='Strong words')
    # display(_df[s]['Word'].to_list())
    wd_list = _df['Word'].to_list()
    
    _df = pd.read_csv(config.en_profanity_datafile, index_col=None)
    s = _df['severity_description'] == 'Severe'
    # wd_list.extend(_df[s]['text'].to_list())
    wd_list.extend(_df['text'].to_list())
    wd_list = set(map(str.lower, wd_list))
    return wd_list
    
offensive_wd_list = get_offensive_words()

In [14]:
def find_phrases(tokens, phrases):
    tokens = list(map(str.lower, tokens))
    """
    Find phrases in a list of sequential tokens.
    
    Args:
        tokens (list): List of sequential tokens.
        phrases (list): List of phrases to search for.
        
    Returns:
        A list of tuples containing the start and end index of each found phrase.
    """
    found_phrases = []
    
    for i in range(len(tokens)):
        for phrase in phrases:
            if tokens[i:i+len(phrase)] == phrase:
                found_phrases.append((i, i+len(phrase)-1))
    
    return found_phrases

def replace_phrases(tk, found_phrases):
    # print(found_phrases)
    tk_out = tk[:]
    off_phrases = []
    for m in found_phrases:
        tk_out[m[1]] = tk[m[1]] + tk[m[1]][-1]
        off_phrases.append(tk[m[0]:m[1]+1])
    return tk_out, off_phrases


def preprocess(t):
    tk = TweetTokenizer()
    detk = mosestokenizer.MosesDetokenizer('en')
    tk = tk.tokenize(t)
    # print(tk)
    phrase_index = find_phrases(tk, list(map(str.split, offensive_wd_list)))
    tk_output = replace_phrases(tk, phrase_index)
    # display(t)
    # fix \n error in tokenizer patch started
    tk_output = list(tk_output)
    tk_output[0] = list(map(lambda x:x.replace('\n', ''), tk_output[0]))
    tk_output = tuple(tk_output)
    # fix \n error in tokenizer patch finished
    return detk(tk_output[0]), phrase_index, tk_output[1]

t, m, o = preprocess("@goll what a Mother fucker, and ass-fuckers")
t, m, o

('@goll what a Mother fuckerr, and ass-fuckerss',
 [(3, 4), (4, 4), (7, 7)],
 [['Mother', 'fucker'], ['fucker'], ['ass-fuckers']])

In [15]:
print(df['Label'].value_counts())
problematic_df = df[df['Label']>0]
problematic_df.shape

Label
0    44874
1     9669
2     1117
Name: count, dtype: int64


(10786, 3)

In [16]:
gtp_dict = {}
gpt_response_file = config.gpt_response_lshs_file
out_file = open(gpt_response_file, "r")
gtp_dict = json.load(out_file)
out_file.close()

In [17]:
def chatgpt_prompt(msg):
    openai.api_key = config.api_key
    messages = [ {"role": "system", "content":
                  "You are a intelligent assistant."},
               ]
    message = msg
    messages.append({"role": "user", "content": message})
    messages.append({"role": "assistant", "content": 'You reply in JSON only, no free text.'})
    # print(messages)
    chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=2048, top_p=1, frequency_penalty=0, presence_penalty=0)
    reply = chat.choices[0].message.content
    return reply, chat

In [24]:
import time
unit_delay = 3
error_delay = 60 * 10

In [27]:
base_msg = 'Substitute problematic terms in the following tweets with inclusive language and produce a list of a few rephrased versions of the tweet and list down suspected problematic terms:\n\n'

bunch = 5
tot = problematic_df.shape[0]
# tot = 400
while True:
    try:
        for i in range(0, tot, bunch):
            if str(i) in gtp_dict:
                continue
            print(i)
            time.sleep(unit_delay)
            message = base_msg
            for j in range(i, min(tot, i+bunch)):
                t = problematic_df.iloc[j].to_list()[1]
                # print(t)
                twt, m, o = preprocess(t)
                message += str(j) +'. ' + twt + '\n'
            # print(message)
            reply, chat = chatgpt_prompt(message)
            gtp_dict[str(i)] = reply
            out_file = open(gpt_response_file, "w")
            json.dump(gtp_dict, out_file, indent = 2)
            out_file.close()
        break
    except:
        print('error')
        time.sleep(error_delay)
        pass

5000
5005
5010
5015
5020
5025
5030
5035
5040
5045
5050
5055
5060
5065
5070
5075
5080
5085
5090
5095
5100
5105
5110
5115
5120
5125
5130
5135
5140
5145
5150
5155
5160
5165
5170
5175
5180
5185
5190
5195
5200
5205
5210
5215
5220
5225
5230
5235
5240
5245
5250
5255
5260
5265
5270
5275
5280
5285
5290
5295
5300
5305
5310
5315
5320
5325
5330
5335
5340
5345
5350
5355
5360
5365
5370
5375
5380
5385
5390
5395
5400
5405
5410
5415
5420
5425
5430
5435
5440
5445
5450
5455
5460
5465
5470
5475
5480
5485
5490
5495
5500
5505
5510
5515
5520
5525
5530
5535
5540
5545
5550
5555
5560
5565
5570
5575
5580
5585
5590
5595
5600
5605
5610
5615
5620
5625
5630
5635
5640
5645
5650
5655
error
5655
5660
5665
5670
5675
5680
5685
5690
5695
5700
5705
5710
5715
5720
5725
5730
5735
5740
5745
5750
5755
5760
5765
5770
5775
5780
5785
5790
5795
5800
5805
5810
5815
5820
5825
5830
5835
5840
5845
5850
5855
5860
5865
5870
5875
5880
5885
5890
5895
5900
5905
5910
5915
5920
5925
5930
5935
5940
5945
5950
5955
5960
5965
5970
5975
5980
5985